<a href="https://colab.research.google.com/github/kregier/AudioLanguageClassifer/blob/main/LanguageClassifier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This notebook contains a third attempt at a language classifier model, which includes 3 dense layers, with a 50% dropout layer between the first two layers. However, the kernel crashes when I try to train this model, and I don't have any idea why. 

This classifier identifies the native language of the speaker from an English reading.

Only the top 10 langauges (in addition to English) from the Speech Sample archive are included.


## Set up the environment and load the metadata

In [1]:
# Set up the environment
!pip install soundfile

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import IPython.display as ipd
import librosa
import soundfile as sf

import os
import random
import re

import tensorflow as tf
import tensorflow_hub as hub

from keras.layers import Dense, Input, Flatten, Dropout
from keras.models import Sequential

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

print("All set up!")

All set up!


In [2]:
#https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [3]:
# Set up the data import using Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

# Change working directory
%cd /content/gdrive/My Drive/Kaggle
!ls

/content/gdrive/My Drive/Kaggle
augment  kaggle.json  processed.csv	   recordings
data	 model	      reading-passage.txt  speakers_all.csv


In [5]:
# Import custom functions that I wrote
import augment
from augment import Augment

from imp import reload

Module imported
Augment scripts reloaded


In [6]:
# Set constants
SAMP_RATE = 16000  #Defined in augment package
BATCH_SIZE = 32  #Defined in augment package
CLF = 'lang10'

In [7]:
meta = pd.read_csv('processed.csv', index_col='speakerid')
meta.head()

,age,age_onset,birthplace,filename,native_language,sex,country,file_missing?
speakerid,,,,,,,,
1,27.0,9.0,"virginia, south africa",afrikaans1,afrikaans,female,south africa,False
2,40.0,5.0,"pretoria, south africa",afrikaans2,afrikaans,male,south africa,False
3,25.0,15.0,"diekabo, ivory coast",agni1,agni,male,ivory coast,False
4,19.0,6.0,"prishtina, kosovo",albanian1,albanian,male,kosovo,False
5,33.0,15.0,"tirana, albania",albanian2,albanian,male,albania,False


# Create a data generator and generate datasets

## Load the VGGish model
Needs to be instantiated before dataset generation can be done.

In [8]:
# Using a SavedModel from the TFHub in Keras
# https://www.tensorflow.org/hub/tf2_saved_model
# VGGish model, from https://tfhub.dev/google/vggish/1

# Link to the model on TFHub
hub_url = 'https://tfhub.dev/google/vggish/1'

# Load the model as a Keras model
vggish_model = hub.KerasLayer(hub_url)
vggish_model.trainable = False

## Create lists of filenames and paths to feed the generators
Following the example from https://keras.io/examples/audio/speaker_recognition_using_cnn/

In [9]:
# Dataset generation from Keras.IO example
# https://keras.io/examples/audio/speaker_recognition_using_cnn/

from pathlib import Path
# Get the list of audio file paths along with their corresponding labels
DATASET_ROOT = "data/lang10"
TRAIN_SUBFOLDER = "train"
VAL_SUBFOLDER = "validation"
TEST_SUBFOLDER = "test/test_data"

DATASET_TRAIN_PATH = os.path.join(DATASET_ROOT, TRAIN_SUBFOLDER)
DATASET_VAL_PATH = os.path.join(DATASET_ROOT, VAL_SUBFOLDER)
DATASET_TEST_PATH = os.path.join(DATASET_ROOT, TEST_SUBFOLDER)

SHUFFLE_SEED = 38
SAMP_RATE = 16000

class_names = os.listdir(DATASET_TRAIN_PATH)
print("Our class names: {}".format(class_names,))

Our class names: ['spanish', 'arabic', 'mandarin', 'french', 'korean', 'russian', 'portuguese', 'dutch', 'turkish', 'german', 'english']


In [10]:
# Process training data
train_paths = []
train_labels = []
for label, name in enumerate(class_names):
    print("Processing language {}".format(name,))
    dir_path = Path(DATASET_TRAIN_PATH) / name
    language_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    train_paths += language_sample_paths
    train_labels += [label] * len(language_sample_paths)

print(
    "Found {} files belonging to {} classes.".format(len(train_paths), len(class_names))
)

Processing language spanish
Processing language arabic
Processing language mandarin
Processing language french
Processing language korean
Processing language russian
Processing language portuguese
Processing language dutch
Processing language turkish
Processing language german
Processing language english
Found 1929 files belonging to 11 classes.


In [11]:
print(train_labels[:2*BATCH_SIZE])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [12]:
# Process validation data
validation_paths = []
val_labels = []
for label, name in enumerate(class_names):
    print("Processing language {}".format(name,))
    dir_path = Path(DATASET_VAL_PATH) / name
    language_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    validation_paths += language_sample_paths
    val_labels += [label] * len(language_sample_paths)

print(
    "Found {} files belonging to {} classes.".format(len(validation_paths), len(class_names))
)

Processing language spanish
Processing language arabic
Processing language mandarin
Processing language french
Processing language korean
Processing language russian
Processing language portuguese
Processing language dutch
Processing language turkish
Processing language german
Processing language english
Found 652 files belonging to 11 classes.


In [13]:
# Process testing data
testing_filenames = os.listdir(DATASET_TEST_PATH)
testing_paths = []
test_labels = []

label_dict = {}
for label, name in enumerate(class_names):
  label_dict[name] = label

for filename in testing_filenames:
  filepath =  os.path.join(DATASET_TEST_PATH, filename)
  testing_paths.append(filepath)
  fname = filename.split('.')[0].rstrip('0123456789')
  test_labels.append(label_dict[fname])

In [14]:
# Shuffle the order of the traning and validation datasets and labels
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(train_paths)
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(train_labels)

rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(validation_paths)
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(val_labels)

rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(testing_paths)
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(test_labels)

## One-hot encode the labels.

In [15]:
train_onehot = tf.keras.backend.one_hot(train_labels, num_classes = len(class_names))
val_onehot = tf.keras.backend.one_hot(val_labels, num_classes = len(class_names))
test_onehot = tf.keras.backend.one_hot(test_labels, num_classes = len(class_names))

print(type(train_onehot) )
print(train_onehot.shape)
print(train_onehot[0])

<class 'tensorflow.python.framework.ops.EagerTensor'>
(1929, 11)
tf.Tensor([1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(11,), dtype=float32)


## Get sample sizes and number of batches for each dataset

In [16]:
# Print sizes of data splits
print("Number of training samples: ", len(train_paths))
print("Number of validation samples: ", len(validation_paths))
print("Number of testing samples: ", len(testing_paths))

Number of training samples:  1929
Number of validation samples:  652
Number of testing samples:  446


In [17]:
# Calculate how many dataset batches to generate
train_steps = np.int(np.ceil(len(train_paths)/BATCH_SIZE)) -1
val_steps = np.int(np.ceil(len(validation_paths)/BATCH_SIZE))-1
eval_steps = np.int(np.ceil(len(testing_paths)/BATCH_SIZE))-1

print("training_steps_per_epoch = ", train_steps)
print("validation_steps = ", val_steps)
print("evaluation_steps = ", eval_steps)

training_steps_per_epoch =  60
validation_steps =  20
evaluation_steps =  13


## Define dataset generator
Then, run a small test dataset, to confirm input dimensions.

In [18]:
def tf_data_generator(file_list, label_list, batch_size=32):
    """ Create a dataset generator. 
    Iterate through a list of filenames and process in batches.
    Extract audio features from vggish model.
    WARNING: This generator forms an infinite loop, 
    so you need to specify how long to run the generator 
    before fitting and evaluating a model.

    Arguments:
    file_list - list of filenames to iterate
    label_list - list of the labels associated with the files
    batch_size - how many files to process at a time
    """
    i = 1
    shuff_seed = 42
    while True: #infinite loop
        if i*batch_size >= len(file_list):
            i=1
            rng = np.random.RandomState(shuff_seed)
            rng.shuffle(file_list)
            rng = np.random.RandomState(shuff_seed)
            rng.shuffle(label_list)
            shuff_seed += 1
        else:
            file_chunk = file_list[(i-1)*batch_size:i*batch_size]
            labels = label_list[(i-1)*batch_size:i*batch_size, :]
            data = []

            for file in file_chunk:
                # Read data
                audio, sr = librosa.load(file, sr=16000)
                # Apply transformations
                embed = vggish_model(audio)
                data.append(embed)
                # Extract labels from filename

            data = np.asarray(data)
            labels = np.asarray(labels)

            yield data, labels
            i += 1

In [19]:
dataset_check = tf.data.Dataset.from_generator(tf_data_generator,
                                         args = [train_paths[:2*BATCH_SIZE], train_onehot[:2*BATCH_SIZE], BATCH_SIZE],
                                         output_types=(tf.float32, tf.float32),
                                         output_shapes= ((None, 10, 128),(None,11)) )

In [20]:
# Check shape and size of dataset batches
for data, labels in dataset_check.take(2):
  print(data.shape)
  print(labels.shape)

(32, 10, 128)
(32, 11)
(32, 10, 128)
(32, 11)


## Generate the datasets for the model

In [21]:
train_dataset = tf.data.Dataset.from_generator(tf_data_generator, 
                                         args = [train_paths, train_onehot, BATCH_SIZE],
                                         output_types=(tf.float32, tf.float32),
                                         output_shapes= ((None, 10, 128),(None,11)) ) 
validation_dataset = tf.data.Dataset.from_generator(tf_data_generator, 
                                         args = [validation_paths, val_onehot, BATCH_SIZE],
                                         output_types=(tf.float32, tf.float32),
                                         output_shapes= ((None, 10, 128),(None,11)) )
test_dataset = tf.data.Dataset.from_generator(tf_data_generator, 
                                         args = [testing_paths, test_onehot, BATCH_SIZE],
                                         output_types=(tf.float32, tf.float32),
                                         output_shapes= ((None, 10, 128),(None,11)) ) 

# Model 3 - 3 dense layers, with dropout

A dropout layer has been added after the first dense layer, in an attempt to avoid overfitting.

## Build the model

In [22]:
from keras.layers import Dropout
def layer3_drop():
  model = Sequential()
  #Input shape (None, 10, 128)
  model.add(Input(shape=(10, 128), batch_size=BATCH_SIZE )) 
  model.add(Dense(128, activation = 'relu' ) ) #, input_shape=(10, 128) ) )
  model.add(Dense(.5) )
  model.add(Dense(64, activation = 'relu' ) ) #, input_shape=(10, 128) ) )
  model.add(Flatten())
  # Output
  model.add(Dense(11, activation='softmax') )

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

lang_3dense_drop = layer3_drop()


Add checkpoint callback to model

In [23]:
ckpt_path = 'model/lang10/checkpoints/lang_3dense_drop.ckpt'
ckpt_dir = os.path.dirname(ckpt_path)
ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath=ckpt_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [24]:
# Add early stopping to train classifier model; default is 10 epochs
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=2)

## Train the model

In [ ]:
# Fit the classifier
#if os.path.isdir('model/lang10/lang_3dense_drop/'):
#  lang_3dense_drop = tf.keras.models.load_model('model/lang10/lang_3dense_drop')
#  hist_3drop_df = pd.read_csv('model/lang10/lang_3dense_drop.history.csv')
#else:
#  if os.path.isfile(ckpt_dir):
    # Load model weights from the most recent checkpoint
#    latest = tf.train.latest_checkpoint(ckpt_dir)
#    lang_3dense_drop.load_weights(latest)
hist_3drop = lang_3dense_drop.fit(train_dataset, 
                        steps_per_epoch=train_steps, 
                        epochs=20, 
                        validation_data=validation_dataset, 
                        validation_steps = val_steps,
                        callbacks=[early_stopping_monitor, ckpt]) #, 
                        #batch_size=BATCH_SIZE)

Epoch 1/20


In [ ]:
# Save the trained model and model history for use later
lang_3dense_drop.save('model/lang10/lang_3dense_drop')

hist_3drop_df = pd.DataFrame(hist_3drop.history) 

# save to csv: 
hist_csv_file = 'model/lang10/lang_3dense_drop.history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_3dense_df.to_csv(f)

## Trained model summary information

In [ ]:
lang_3dense_drop.summary()

In [ ]:
hist_3drop_df.head()

In [ ]:
plt.plot(hist_3drop_df['accuracy'])
plt.plot(hist_3drop_df['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(hist_3drop_df['loss'])
plt.plot(hist_3drop_df['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Evaluate the trained classifier
Based on the methods used fro the Gender classifier; probably need to be refined, but let's look for now.

In [ ]:
# Evaluate on validation set
val_loss_3dr, val_acc_3dr = lang_3dense_drop.evaluate(validation_dataset, steps=val_steps)

In [ ]:
# Evaluate on testing set
test_loss_3dr, test_acc_3dr = lang_3dense_drop.evaluate(test_dataset, steps=eval_steps)

In [ ]:
# Predict testing set
y_pred_3dr = lang_3dense_drop.predict(test_dataset, steps=eval_steps)
print(y_pred_3dr.shape)

In [ ]:
print(y_pred_3dr[:2])

In [ ]:
# Get label index using np.argmax from each prediction
y_pred_ind_3dr = np.argmax(y_pred, axis=1)
print(y_pred_ind_3dr[:5])

In [ ]:
# Get np.argmax from each prediction as label prediction

In [ ]:
con_matrix_3dr = tf.math.confusion_matrix(test_labels[:y_pred_3dr.shape[0]], y_pred_ind_3dr)

In [ ]:
sns.heatmap(con_matrix_3dr, annot = True, cmap='YlGnBu')

# Other model building steps:
- Add drop out layers
- Add more dense layers


What is a good accuracy rate for multi-class classification?

